# 运行先安MNE，pip install mne
# 解压文件

In [1]:
# !unzip "data/data112870/BCICIV_2a_mat.zip" -d data/

In [2]:
from read_data import read_data
import numpy as np

In [3]:
train_path = r"data/A01T.mat"
test_path = r"data/A01E.mat"

# Trian Data

In [4]:
train_data=np.zeros((6,1056,751),dtype=np.float32)
train_label=np.zeros((6,1056),dtype=np.int64)
for i in range(6):
    data,label = read_data(train_path,-(i+1))
    train_data[i]=data
    label = np.array(label[:])
    train_label[i]=label


Creating RawArray with float64 data, n_channels=25, n_times=96735
    Range : 0 ... 96734 =      0.000 ...   386.936 secs
Ready.
Not setting metadata
Not setting metadata
48 matching events found
Setting baseline interval to [3.0, 6.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 48 events and 751 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=25, n_times=96735
    Range : 0 ... 96734 =      0.000 ...   386.936 secs
Ready.
Not setting metadata
Not setting metadata
48 matching events found
Setting baseline interval to [3.0, 6.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 48 events and 751 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=25, n_times=96735
    Range : 0 ... 96734 =      0.000 ...   386.936 secs
Ready.
Not setting metadata
Not setting metadata
48 matching events found
Setting baseline int

# Test Data

In [5]:
# test_data=np.zeros((6,1056,751),dtype=np.float32)
# test_label=np.zeros((6,1056),dtype=np.int64)
# for i in range(6):
#     data,label = read_data(test_path,-(i+1))
#     test_data[i]=data
#     label = np.array(label[:])
#     test_label[i]=label


# Data_Reshape

In [6]:
train_data = train_data.reshape(6336,751)
train_label = train_label.reshape(6336,)
# test_data = test_data.reshape(6336,751)
# test_label = test_label.reshape(6336,)
print(train_data[0][0])

-4.29707


# 验证标签的对错

In [ ]:
import paddle
import paddle.nn.functional as F
import paddle.nn as nn


In [ ]:
class CNN_GRU(nn.Layer):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1D(1,32,9,1,padding='same',data_format='NCL')
        self.padding1 = nn.MaxPool1D(kernel_size=2,stride=2,padding='valid')
        self.conv2 = nn.Conv1D(32,32,9,1,padding='same',data_format='NCL')
        self.padding2 = nn.MaxPool1D(kernel_size=2,stride=2,padding='valid')
        self.conv3 = nn.Conv1D(32,32,9,1,padding='same',
        data_format='NCL')
        self.padding3 = nn.MaxPool1D(kernel_size=2,stride=2,padding='valid')
        self.flatten1 = nn.Flatten()
        self.flatten2 = nn.Flatten()
        # self.gru = nn.GRU(1,64,1)
        self.gru = nn.GRU(2976,64,1)
        self.dense1 = nn.Linear(64,64)
        self.dense2 = nn.Linear(64,4)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.relu3 = nn.ReLU()
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
    def forward(self,x):
        x=self.conv1(x)
        x=self.relu1(x)
        x=self.padding1(x)
        x=self.conv2(x)
        x=self.relu2(x)
        x=self.padding2(x)
        x=self.conv3(x)
        x=self.relu3(x)
        x=self.padding3(x)
        x=self.flatten1(x)
        # x = x.unsqueeze(-1)
        x = x.unsqueeze(1)
        x,h=self.gru(x)
        x=self.dense1(x)
        x=self.relu4(x)
        x=self.dropout(x)
        x=self.dense2(x)
        x=self.flatten2(x)
        return x

In [ ]:
input_spec = paddle.static.InputSpec(
    shape=(-1,1,751),
    dtype='float32',
    name='x'
)
model = CNN_GRU()

paddle.summary(model,input_spec)
out = model(paddle.randn((1,1,750)))
print(out)

-------------------------------------------------------------------------------
 Layer (type)       Input Shape            Output Shape           Param #    
   Conv1D-1        [[1, 1, 751]]           [1, 32, 751]             320      
    ReLU-1         [[1, 32, 751]]          [1, 32, 751]              0       
  MaxPool1D-1      [[1, 32, 751]]          [1, 32, 375]              0       
   Conv1D-2        [[1, 32, 375]]          [1, 32, 375]            9,248     
    ReLU-2         [[1, 32, 375]]          [1, 32, 375]              0       
  MaxPool1D-2      [[1, 32, 375]]          [1, 32, 187]              0       
   Conv1D-3        [[1, 32, 187]]          [1, 32, 187]            9,248     
    ReLU-3         [[1, 32, 187]]          [1, 32, 187]              0       
  MaxPool1D-3      [[1, 32, 187]]          [1, 32, 93]               0       
   Flatten-1       [[1, 32, 93]]            [1, 2976]                0       
     GRU-1         [[1, 1, 2976]]    [[1, 1, 64], [1, 1, 64]] 

In [ ]:
loss_function = paddle.nn.CrossEntropyLoss()
# lr_scheduler = paddle.optimizer.lr.CosineAnnealingDecay(
#     learning_rate=LEARNING_RATE,
#     T_max=NUM_EPOCHS
# )
optimizer = paddle.optimizer.Adam(
    learning_rate=1e-4,
    parameters=model.parameters()
)

# 8:2 T

In [ ]:
data = train_data
label = train_label-1
print(data.shape,label.shape)

from sklearn.preprocessing import StandardScaler #标准化
from sklearn.preprocessing import MinMaxScaler
sk = StandardScaler()
data = sk.fit_transform(data)

from sklearn.model_selection import train_test_split
data=np.expand_dims(data,1)
label=np.expand_dims(label,1)
# train_data,test_data,train_label,test_label = train_test_split(data,label,train_size=0.8)



(6336, 751) (6336,)


# 8:2 E

In [ ]:
# data = test_data
# label = test_label-1
# print(data.shape,label.shape)

# from sklearn.preprocessing import StandardScaler #标准化
# from sklearn.preprocessing import MinMaxScaler
# sk = StandardScaler()
# data = sk.fit_transform(data)
# from sklearn.model_selection import train_test_split
# label=label
# data=np.expand_dims(data,1)
# label=np.expand_dims(label,1)
## train_data,test_data,train_label,test_label = train_test_split(data,label,train_size=0.8)

# E.T

In [ ]:
# train_label=train_label-1
# test_label=test_label-1
# from sklearn.preprocessing import StandardScaler #标准化
# from sklearn.preprocessing import MinMaxScaler
# sk = StandardScaler()
# train_data = sk.fit_transform(train_data)
# test_data = sk.fit_transform(test_data)
# train_data=np.expand_dims(train_data,1)
# train_label=np.expand_dims(train_label,1)
# test_data=np.expand_dims(test_data,1)
# test_label=np.expand_dims(test_label,1)

In [ ]:
NUM_EPOCHS=100
TRAIN_BATCH_SIZE=28
# print(train_label.shape,test_label.shape)
# print(train_data.shape,test_data.shape)

# 五折交叉验证

In [ ]:
def k_fold_indices(num_samples, k=5):
    indices = np.random.permutation(num_samples)
    if k <= 1:
        print(f"Expect k>1, but received k={k}")
    n_samples_per_fold = num_samples // k

    anchors = [n_samples_per_fold for _ in range(k)]
    for i in range(0, num_samples - k * n_samples_per_fold):
        anchors[i] += 1
    for i in range(1, len(anchors)):
        anchors[i] = anchors[i - 1] + anchors[i]
    assert anchors[-1] == num_samples
    anchors.pop(-1)
    assert len(anchors) == k - 1
    
    folds = np.split(indices, anchors)
    train_indices = [np.empty(0, dtype=np.int64) for _ in range(k)]
    valid_indices = []

    for i, fold in enumerate(folds):
        valid_indices.append(fold)
        for j in range(k):
            if j == i:
                continue
            train_indices[j] = np.concatenate((train_indices[j], fold))

    return list(zip(train_indices, valid_indices))


In [ ]:
import numpy as np
from paddle.io import Dataset


class MIDataset(Dataset):
    def __init__(self, data,label):
        self.data = data
        self.label = label

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.label[idx]
        return data, label

    def __len__(self):
        return len(self.label)

In [ ]:
result=[]
for train_indices, valid_indices in k_fold_indices(data.shape[0], 5):
    train_data = data[train_indices]
    train_label = label[train_indices]
    test_data = data[valid_indices]
    test_label = label[valid_indices]
    train_dataset = MIDataset(train_data,train_label)
    test_dataset = MIDataset(test_data,test_label)
    train_loader = paddle.io.DataLoader(
        train_dataset, 
        batch_size=28, 
        shuffle=True,
        drop_last=True)
    test_loader = paddle.io.DataLoader(
        test_dataset, 
        batch_size=28,
        shuffle=True
    )
    model = CNN_GRU()
    loss_function = paddle.nn.CrossEntropyLoss()
    optimizer = paddle.optimizer.Adam(
    learning_rate=1e-4,
    parameters=model.parameters())
    model_handler = paddle.Model(model)
    model_handler.prepare(optimizer=optimizer,
              loss=loss_function,
              metrics=paddle.metric.Accuracy())
    model_handler.fit(train_loader,
          test_loader,
          epochs=100,
          batch_size=28,
          verbose=1)
    result.append(model_handler.evaluate(test_dataset, batch_size=test_data.shape[0]))
a=0
for i in range(5):
    a=result[i]['acc']/5+a
a

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/100
step  20/181 [==>...........................] - loss: 1.3826 - acc: 0.2786 - ETA: 2s - 15ms/step

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 181/181 [==============================] - loss: 1.2563 - acc: 0.3254 - 8ms/step          
Eval begin...
step 46/46 [==============================] - loss: 1.0529 - acc: 0.5055 - 4ms/step          
Eval samples: 1268
Epoch 2/100
step 181/181 [==============================] - loss: 1.1867 - acc: 0.4773 - 7ms/step          
Eval begin...
step 46/46 [==============================] - loss: 1.1042 - acc: 0.6569 - 5ms/step          
Eval samples: 1268
Epoch 3/100
step 181/181 [==============================] - loss: 0.8814 - acc: 0.6150 - 8ms/step          
Eval begin...
step 46/46 [==============================] - loss: 0.8707 - acc: 0.8021 - 4ms/step          
Eval samples: 1268
Epoch 4/100
step 150/181 [=======================>......] - loss: 0.5813 - acc: 0.7350 - ETA: 0s - 7ms/step